In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
from utils import PkdbModel, caffeine_idx, pktype_data, filter_out, convert_unit, unstring
plt.style.use('seaborn-whitegrid')
import numpy as np

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
def to_numeric(df_data):
    df_data[("weight","value")] = pd.to_numeric(df_data[("weight","value")])
    df_data[("weight","mean")] = pd.to_numeric(df_data[("weight","mean")])
    df_data["value_intervention"] = pd.to_numeric(df_data["value_intervention"])
    df_data["median"] = pd.to_numeric(df_data["median"])
    df_data["mean"] = pd.to_numeric(df_data["mean"])
    df_data["value"] = pd.to_numeric(df_data["value"])
    df_data["se"] = pd.to_numeric(df_data["se"])
    df_data["sd"] = pd.to_numeric(df_data["sd"])
    df_data["cv"] = pd.to_numeric(df_data["cv"])
    return df_data
     
def invere_from_interventions(data, unit_abs = "mg"):
    unit_rel= unit_abs + "/kg"
    data_rel = convert_unit(data,
                             unit_in=unit_abs,
                             unit_out=unit_rel, 
                             factor=1.0/data[("weight","value")],
                             unit_field="unit_intervention", 
                             data_fields=['value_intervention'], 
                             subset=[("weight","value"),"value"])

    data_abs = convert_unit(data,
                             unit_in=unit_rel,
                             unit_out=unit_abs, 
                             factor=data[("weight","value")],
                             unit_field="unit_intervention", 
                             data_fields=['value_intervention'],
                             subset=[("weight","value"),"value"])
    
    data = pd.concat([data,data_rel,data_abs], ignore_index=True)
    
    data_rel = convert_unit(data,
                             unit_in=unit_abs,
                             unit_out=unit_rel, 
                             factor=1.0/data[("weight","mean")],
                             unit_field="unit_intervention", 
                             data_fields=['value_intervention'], 
                             subset=[("weight","mean"),"mean"])

    data_abs = convert_unit(data,
                             unit_in=unit_rel,
                             unit_out=unit_abs, 
                             factor=data[("weight","mean")],
                             unit_field="unit_intervention", 
                             data_fields=['value_intervention'],
                             subset=[("weight","mean"),"mean"])
    
    data = pd.concat([data,data_rel,data_abs], ignore_index=True)
    
    return data

def invere_from_outputs(data, unit_abs="l/h"):
    unit_rel= unit_abs + "/kg"
    data_rel = convert_unit(data,
                                 unit_in=unit_abs,
                                 unit_out=unit_rel, 
                                 factor=1.0/data[("weight","value")],
                                 unit_field="unit", 
                                 data_fields=['value'], 
                                 subset=[("weight","value"),"value"])

    data_abs_i = convert_unit(data,
                                 unit_in=unit_rel,
                                 unit_out=unit_abs, 
                                 factor=data[("weight","value")],
                                 unit_field="unit", 
                                 data_fields=['value'],
                                 subset=[("weight","value"),"value"])
    

    data = pd.concat([data,data_rel,data_abs_i], ignore_index=True)
    
    data_rel = convert_unit(data,
                             unit_in=unit_abs,
                             unit_out=unit_rel, 
                             factor=1.0/data[("weight","mean")],
                             unit_field="unit", 
                             data_fields=['mean','median','sd', 'se', "cv"],
                             subset=[("weight","mean"),"mean"])

    data_abs = convert_unit(data,
                             unit_in=unit_rel,
                             unit_out=unit_abs, 
                             factor=data[("weight","mean")],
                             unit_field="unit", 
                             data_fields=['mean','median','sd', 'se', "cv"],
                             subset=[("weight","mean"),"mean"])


    data = pd.concat([data,data_rel,data_abs], ignore_index=True)
    
    return data

In [4]:
all_complete = PkdbModel("all_complete", destination="2-merged")
all_complete.read()
all_complete.data.loc[(all_complete.data[("sex","choice")] == "M"),("oral contraceptives","choice")] = "N"
complete_caffeine = all_complete.data[caffeine_idx(all_complete.data)]

/home/janekg89/Envs/pkdb_analysis/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2903: DtypeWarning: Columns (21,22,23,33,47,60,77,79,97,99,101,106,110,114,120,121,125,133,134,135,143,152,153,155,158,161,162,164,165,166,167,168,176,179,180,181,189,196,197,204,209,214,219,224,232,238,239,246,252,258,268) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [5]:
caffeine_clearance = pktype_data(complete_caffeine,"clearance")
caffeine_clearance = filter_out(caffeine_clearance,"unit",["ml/min/1.73m^2"])
caffeine_clearance = to_numeric(caffeine_clearance)
caffeine_clearance = invere_from_interventions(caffeine_clearance,"mg")
caffeine_clearance = invere_from_outputs(caffeine_clearance,"l/h")

In [6]:
caffeine_clr = PkdbModel("caffeine_clearance",destination="3-inferred")
caffeine_clr.data = caffeine_clearance
caffeine_clr.save()

## Timecourses

In [7]:
caffeine_timecourse = pktype_data(complete_caffeine,"concentration")
caffeine_timecourse = filter_out(caffeine_timecourse,"output_type",["outputs"])
caffeine_timecourse = filter_out(caffeine_timecourse,"unit",["µmol/l"])
caffeine_timecourse.groupby(["unit","unit_intervention"]).count()

study  output_type   pk  tissue  substance_name  \
unit  unit_intervention                                                    
µg/ml mg                   103          103  103     103             103   
      mg/kg                  9            9    9       9               9   

                         pktype  interventions  individual_pk  value  \
unit  unit_intervention                                                
µg/ml mg                    103            103             86     86   
      mg/kg                   9              9              5      5   

                         group_pk         ...          \
unit  unit_intervention                   ...           
µg/ml mg                       17         ...           
      mg/kg                     4         ...           

                         (metabolic ratio, count)  (metabolic ratio, max)  \
unit  unit_intervention                                                     
µg/ml mg                                        0                       0   
      mg/kg                                     0                       0   

                         (metabolic ratio, mean)  (metabolic ratio, min)  \
unit  unit_intervention                                                    
µg/ml mg                                       0                       0   
      mg/kg                                    0                       0   

                         (metabolic ratio, unit)  (fat weight, count)  \
unit  unit_intervention                                                 
µg/ml mg                                       0                    0   
      mg/kg                                    0                    0   

                         (fat weight, cv)  (fat weight, mean)  \
unit  unit_intervention                                         
µg/ml mg                                0                   0   
      mg/kg                             0                   0   

                         (fat weight, se)  (fat weight, unit)  
unit  unit_intervention                                        
µg/ml mg                                0                   0  
      mg/kg                             0                   0  

[2 rows x 266 columns]

In [8]:
caffeine_tc = PkdbModel("caffeine_timecourse",destination="3-inferred")
caffeine_tc.data = caffeine_timecourse
caffeine_tc.to_array()
caffeine_timecourse = caffeine_tc.data

In [9]:
caffeine_timecourse = invere_from_interventions(caffeine_timecourse,"mg")
caffeine_timecourse = invere_from_outputs(caffeine_timecourse,"µg/ml")

In [10]:
caffeine_timecourse.groupby(["unit","unit_intervention","time_unit"]).count()

study  output_type   pk  tissue  \
unit     unit_intervention time_unit                                    
µg/ml    mg                h            105          105  105     105   
         mg/kg             h            102          102  102     102   
µg/ml/kg mg                h             95           95   95      95   
         mg/kg             h             95           95   95      95   

                                      substance_name  pktype  interventions  \
unit     unit_intervention time_unit                                          
µg/ml    mg                h                     105     105            105   
         mg/kg             h                     102     102            102   
µg/ml/kg mg                h                      95      95             95   
         mg/kg             h                      95      95             95   

                                      individual_pk  value  group_pk  \
unit     unit_intervention time_unit                                   
µg/ml    mg                h                     86     86        19   
         mg/kg             h                     89     89        13   
µg/ml/kg mg                h                     84     84        11   
         mg/kg             h                     84     84        11   

                                             ...          \
unit     unit_intervention time_unit         ...           
µg/ml    mg                h                 ...           
         mg/kg             h                 ...           
µg/ml/kg mg                h                 ...           
         mg/kg             h                 ...           

                                      (metabolic ratio, count)  \
unit     unit_intervention time_unit                             
µg/ml    mg                h                                 0   
         mg/kg             h                                 0   
µg/ml/kg mg                h                                 0   
         mg/kg             h                                 0   

                                      (metabolic ratio, max)  \
unit     unit_intervention time_unit                           
µg/ml    mg                h                               0   
         mg/kg             h                               0   
µg/ml/kg mg                h                               0   
         mg/kg             h                               0   

                                      (metabolic ratio, mean)  \
unit     unit_intervention time_unit                            
µg/ml    mg                h                                0   
         mg/kg             h                                0   
µg/ml/kg mg                h                                0   
         mg/kg             h                                0   

                                      (metabolic ratio, min)  \
unit     unit_intervention time_unit                           
µg/ml    mg                h                               0   
         mg/kg             h                               0   
µg/ml/kg mg                h                               0   
         mg/kg             h                               0   

                                      (metabolic ratio, unit)  \
unit     unit_intervention time_unit                            
µg/ml    mg                h                                0   
         mg/kg             h                                0   
µg/ml/kg mg                h                                0   
         mg/kg             h                                0   

                                      (fat weight, count)  (fat weight, cv)  \
unit     unit_intervention time_unit                                          
µg/ml    mg                h                            0                 0   
         mg/kg             h                            0                 0   
µg/ml/kg mg                h                            0        

In [11]:
caffeine_tc = PkdbModel("caffeine_timecourse",destination="3-inferred")
caffeine_tc.data = caffeine_timecourse
caffeine_tc.save()

In [12]:
caffeine_thalf = pktype_data(complete_caffeine,"thalf")
caffeine_thalf = to_numeric(caffeine_thalf)
caffeine_thalf.groupby(["unit","unit_intervention"]).count()


/home/janekg89/Envs/pkdb_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/janekg89/Envs/pkdb_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/janekg89/Envs/pkdb_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

study  output_type   pk  tissue  substance_name  \
unit unit_intervention                                                    
h    mg                   411          411  411     411             411   
     mg/kg                 60           60   60      60              60   

                        pktype  interventions  individual_pk  value  group_pk  \
unit unit_intervention                                                          
h    mg                    411            411            345    345        66   
     mg/kg                  60             60             36     36        24   

                               ...          (metabolic ratio, count)  \
unit unit_intervention         ...                                     
h    mg                        ...                                 0   
     mg/kg                     ...                                 0   

                        (metabolic ratio, max)  (metabolic ratio, mean)  \
unit unit_intervention                                                    
h    mg                                      0                        0   
     mg/kg                                   0                        0   

                        (metabolic ratio, min)  (metabolic ratio, unit)  \
unit unit_intervention                                                    
h    mg                                      0                        0   
     mg/kg                                   0                        0   

                        (fat weight, count)  (fat weight, cv)  \
unit unit_intervention                                          
h    mg                                   0                 0   
     mg/kg                                8                 8   

                        (fat weight, mean)  (fat weight, se)  \
unit unit_intervention                                         
h    mg                                  0                 0   
     mg/kg                               8                 8   

                        (fat weight, unit)  
unit unit_intervention                      
h    mg                                  0  
     mg/kg                               8  

[2 rows x 266 columns]

In [13]:
caffeine_thalf = invere_from_interventions(caffeine_thalf,"mg")

In [14]:
caffeine_thalf.groupby(["unit","unit_intervention"]).count()

study  output_type   pk  tissue  substance_name  \
unit unit_intervention                                                    
h    mg                   457          457  457     457             457   
     mg/kg                349          349  349     349             349   

                        pktype  interventions  individual_pk  value  group_pk  \
unit unit_intervention                                                          
h    mg                    457            457            369    369        88   
     mg/kg                 349            349            286    286        63   

                               ...          (metabolic ratio, count)  \
unit unit_intervention         ...                                     
h    mg                        ...                                 0   
     mg/kg                     ...                                 0   

                        (metabolic ratio, max)  (metabolic ratio, mean)  \
unit unit_intervention                                                    
h    mg                                      0                        0   
     mg/kg                                   0                        0   

                        (metabolic ratio, min)  (metabolic ratio, unit)  \
unit unit_intervention                                                    
h    mg                                      0                        0   
     mg/kg                                   0                        0   

                        (fat weight, count)  (fat weight, cv)  \
unit unit_intervention                                          
h    mg                                   8                 8   
     mg/kg                                8                 8   

                        (fat weight, mean)  (fat weight, se)  \
unit unit_intervention                                         
h    mg                                  8                 8   
     mg/kg                               8                 8   

                        (fat weight, unit)  
unit unit_intervention                      
h    mg                                  8  
     mg/kg                               8  

[2 rows x 266 columns]

In [15]:
caffeine_thf = PkdbModel("caffeine_thalf",destination="3-inferred")
caffeine_thf.data = caffeine_thalf
caffeine_thf.save()

In [16]:
caffeine_tmax = pktype_data(complete_caffeine,"tmax")
caffeine_tmax = to_numeric(caffeine_tmax)

/home/janekg89/Envs/pkdb_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/janekg89/Envs/pkdb_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/janekg89/Envs/pkdb_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

In [17]:
caffeine_tmax.groupby(["unit","unit_intervention"]).count()

study  output_type   pk  tissue  substance_name  \
unit unit_intervention                                                    
h    mg                   114          114  114     114             114   
     mg/kg                 23           23   23      23              23   

                        pktype  interventions  individual_pk  value  group_pk  \
unit unit_intervention                                                          
h    mg                    114            114             77     77        37   
     mg/kg                  23             23             12     10        11   

                               ...          (metabolic ratio, count)  \
unit unit_intervention         ...                                     
h    mg                        ...                                 0   
     mg/kg                     ...                                 0   

                        (metabolic ratio, max)  (metabolic ratio, mean)  \
unit unit_intervention                                                    
h    mg                                      0                        0   
     mg/kg                                   0                        0   

                        (metabolic ratio, min)  (metabolic ratio, unit)  \
unit unit_intervention                                                    
h    mg                                      0                        0   
     mg/kg                                   0                        0   

                        (fat weight, count)  (fat weight, cv)  \
unit unit_intervention                                          
h    mg                                   0                 0   
     mg/kg                                8                 8   

                        (fat weight, mean)  (fat weight, se)  \
unit unit_intervention                                         
h    mg                                  0                 0   
     mg/kg                               8                 8   

                        (fat weight, unit)  
unit unit_intervention                      
h    mg                                  0  
     mg/kg                               8  

[2 rows x 266 columns]

In [18]:
caffeine_tmax =  invere_from_interventions(caffeine_tmax)

In [19]:
caffeine_tmax.groupby(["unit","unit_intervention"]).count()

study  output_type   pk  tissue  substance_name  \
unit unit_intervention                                                    
h    mg                   123          123  123     123             123   
     mg/kg                 66           66   66      66              66   

                        pktype  interventions  individual_pk  value  group_pk  \
unit unit_intervention                                                          
h    mg                    123            123             77     77        46   
     mg/kg                  66             66             32     30        34   

                               ...          (metabolic ratio, count)  \
unit unit_intervention         ...                                     
h    mg                        ...                                 0   
     mg/kg                     ...                                 0   

                        (metabolic ratio, max)  (metabolic ratio, mean)  \
unit unit_intervention                                                    
h    mg                                      0                        0   
     mg/kg                                   0                        0   

                        (metabolic ratio, min)  (metabolic ratio, unit)  \
unit unit_intervention                                                    
h    mg                                      0                        0   
     mg/kg                                   0                        0   

                        (fat weight, count)  (fat weight, cv)  \
unit unit_intervention                                          
h    mg                                   8                 8   
     mg/kg                                8                 8   

                        (fat weight, mean)  (fat weight, se)  \
unit unit_intervention                                         
h    mg                                  8                 8   
     mg/kg                               8                 8   

                        (fat weight, unit)  
unit unit_intervention                      
h    mg                                  8  
     mg/kg                               8  

[2 rows x 266 columns]

In [20]:
caffeine_tmx = PkdbModel("caffeine_tmax",destination="3-inferred")
caffeine_tmx.data = caffeine_tmax
caffeine_tmx.save()

In [21]:
caffeine_vd = pktype_data(complete_caffeine,"vd")
caffeine_vd = to_numeric(caffeine_vd)

/home/janekg89/Envs/pkdb_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/janekg89/Envs/pkdb_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/janekg89/Envs/pkdb_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

In [22]:
caffeine_vd.groupby(["unit","unit_intervention"]).count()

study  output_type   pk  tissue  substance_name  \
unit unit_intervention                                                    
l    mg                   141          141  141     141             141   
     mg/kg                 11           11   11      11              11   
l/kg mg                   160          160  160     160             160   
     mg/kg                 37           37   37      37              37   

                        pktype  interventions  individual_pk  value  group_pk  \
unit unit_intervention                                                          
l    mg                    141            141            123    123        18   
     mg/kg                  11             11              0      0        11   
l/kg mg                    160            160            120    120        40   
     mg/kg                  37             37             21     21        16   

                               ...          (metabolic ratio, count)  \
unit unit_intervention         ...                                     
l    mg                        ...                                 0   
     mg/kg                     ...                                 0   
l/kg mg                        ...                                 0   
     mg/kg                     ...                                 0   

                        (metabolic ratio, max)  (metabolic ratio, mean)  \
unit unit_intervention                                                    
l    mg                                      0                        0   
     mg/kg                                   0                        0   
l/kg mg                                      0                        0   
     mg/kg                                   0                        0   

                        (metabolic ratio, min)  (metabolic ratio, unit)  \
unit unit_intervention                                                    
l    mg                                      0                        0   
     mg/kg                                   0                        0   
l/kg mg                                      0                        0   
     mg/kg                                   0                        0   

                        (fat weight, count)  (fat weight, cv)  \
unit unit_intervention                                          
l    mg                                   0                 0   
     mg/kg                                0                 0   
l/kg mg                                   0                 0   
     mg/kg                                8                 8   

                        (fat weight, mean)  (fat weight, se)  \
unit unit_intervention                                         
l    mg                                  0                 0   
     mg/kg                               0                 0   
l/kg mg                                  0                 0   
     mg/kg                               8                 8   

                        (fat weight, unit)  
unit unit_intervention                      
l    mg                                  0  
     mg/kg                               0  
l/kg mg                                  0  
     mg/kg                               8  

[4 rows x 266 columns]

In [23]:
caffeine_vd =  invere_from_interventions(caffeine_vd, "mg")
caffeine_vd =  invere_from_outputs(caffeine_vd,"l")

In [24]:
caffeine_vd.groupby(["unit","unit_intervention"]).count()

study  output_type   pk  tissue  substance_name  \
unit unit_intervention                                                    
l    mg                   268          268  268     268             268   
     mg/kg                232          232  232     232             232   
l/kg mg                   299          299  299     299             299   
     mg/kg                246          246  246     246             246   

                        pktype  interventions  individual_pk  value  group_pk  \
unit unit_intervention                                                          
l    mg                    268            268            198    198        70   
     mg/kg                 232            232            177    177        55   
l/kg mg                    299            299            231    231        68   
     mg/kg                 246            246            189    189        57   

                               ...          (metabolic ratio, count)  \
unit unit_intervention         ...                                     
l    mg                        ...                                 0   
     mg/kg                     ...                                 0   
l/kg mg                        ...                                 0   
     mg/kg                     ...                                 0   

                        (metabolic ratio, max)  (metabolic ratio, mean)  \
unit unit_intervention                                                    
l    mg                                      0                        0   
     mg/kg                                   0                        0   
l/kg mg                                      0                        0   
     mg/kg                                   0                        0   

                        (metabolic ratio, min)  (metabolic ratio, unit)  \
unit unit_intervention                                                    
l    mg                                      0                        0   
     mg/kg                                   0                        0   
l/kg mg                                      0                        0   
     mg/kg                                   0                        0   

                        (fat weight, count)  (fat weight, cv)  \
unit unit_intervention                                          
l    mg                                   8                 8   
     mg/kg                                8                 8   
l/kg mg                                   8                 8   
     mg/kg                                8                 8   

                        (fat weight, mean)  (fat weight, se)  \
unit unit_intervention                                         
l    mg                                  8                 8   
     mg/kg                               8                 8   
l/kg mg                                  8                 8   
     mg/kg                               8                 8   

                        (fat weight, unit)  
unit unit_intervention                      
l    mg                                  8  
     mg/kg                               8  
l/kg mg                                  8  
     mg/kg                               8  

[4 rows x 266 columns]

In [25]:
caffeine_vd_model = PkdbModel("caffeine_vd",destination="3-inferred")
caffeine_vd_model.data = caffeine_vd
caffeine_vd_model.save()

In [28]:
caffeine_auc_inf = pktype_data(complete_caffeine,"auc_inf")
caffeine_auc_inf = to_numeric(caffeine_auc_inf)
caffeine_auc_inf.groupby(["unit","unit_intervention"]).count()

/home/janekg89/Envs/pkdb_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/janekg89/Envs/pkdb_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/janekg89/Envs/pkdb_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

study  output_type   pk  tissue  substance_name  \
unit     unit_intervention                                                    
mg*h/l   mg                   209          209  209     209             209   
         mg/kg                 37           37   37      37              37   
µmol*h/l mg                     7            7    7       7               7   

                            pktype  interventions  individual_pk  value  \
unit     unit_intervention                                                
mg*h/l   mg                    209            209            196    196   
         mg/kg                  37             37              0      0   
µmol*h/l mg                      7              7              0      0   

                            group_pk         ...          \
unit     unit_intervention                   ...           
mg*h/l   mg                       13         ...           
         mg/kg                    37         ...           
µmol*h/l mg                        7         ...           

                            (metabolic ratio, count)  (metabolic ratio, max)  \
unit     unit_intervention                                                     
mg*h/l   mg                                        0                       0   
         mg/kg                                     0                       0   
µmol*h/l mg                                        0                       0   

                            (metabolic ratio, mean)  (metabolic ratio, min)  \
unit     unit_intervention                                                    
mg*h/l   mg                                       0                       0   
         mg/kg                                    0                       0   
µmol*h/l mg                                       0                       0   

                            (metabolic ratio, unit)  (fat weight, count)  \
unit     unit_intervention                                                 
mg*h/l   mg                                       0                    0   
         mg/kg                                    0                    8   
µmol*h/l mg                                       0                    0   

                            (fat weight, cv)  (fat weight, mean)  \
unit     unit_intervention                                         
mg*h/l   mg                                0                   0   
         mg/kg                             8                   8   
µmol*h/l mg                                0                   0   

                            (fat weight, se)  (fat weight, unit)  
unit     unit_intervention                                        
mg*h/l   mg                                0                   0  
         mg/kg                             8                   8  
µmol*h/l mg                                0                   0  

[3 rows x 266 columns]

In [30]:
caffeine_auc_inf = filter_out(caffeine_auc_inf,"unit",["µmol*h/l"])
caffeine_auc_inf.groupby(["unit","unit_intervention"]).count()

study  output_type   pk  tissue  substance_name  \
unit   unit_intervention                                                    
mg*h/l mg                   209          209  209     209             209   
       mg/kg                 37           37   37      37              37   

                          pktype  interventions  individual_pk  value  \
unit   unit_intervention                                                
mg*h/l mg                    209            209            196    196   
       mg/kg                  37             37              0      0   

                          group_pk         ...          \
unit   unit_intervention                   ...           
mg*h/l mg                       13         ...           
       mg/kg                    37         ...           

                          (metabolic ratio, count)  (metabolic ratio, max)  \
unit   unit_intervention                                                     
mg*h/l mg                                        0                       0   
       mg/kg                                     0                       0   

                          (metabolic ratio, mean)  (metabolic ratio, min)  \
unit   unit_intervention                                                    
mg*h/l mg                                       0                       0   
       mg/kg                                    0                       0   

                          (metabolic ratio, unit)  (fat weight, count)  \
unit   unit_intervention                                                 
mg*h/l mg                                       0                    0   
       mg/kg                                    0                    8   

                          (fat weight, cv)  (fat weight, mean)  \
unit   unit_intervention                                         
mg*h/l mg                                0                   0   
       mg/kg                             8                   8   

                          (fat weight, se)  (fat weight, unit)  
unit   unit_intervention                                        
mg*h/l mg                                0                   0  
       mg/kg                             8                   8  

[2 rows x 266 columns]

In [31]:
caffeine_auc_inf =  invere_from_interventions(caffeine_auc_inf, "mg")
caffeine_auc_inf =  invere_from_outputs(caffeine_auc_inf,"mg*h/l")

In [32]:
caffeine_auc_inf.groupby(["unit","unit_intervention"]).count()

study  output_type   pk  tissue  substance_name  \
unit      unit_intervention                                                    
mg*h/l    mg                   246          246  246     246             246   
          mg/kg                209          209  209     209             209   
mg*h/l/kg mg                   209          209  209     209             209   
          mg/kg                209          209  209     209             209   

                             pktype  interventions  individual_pk  value  \
unit      unit_intervention                                                
mg*h/l    mg                    246            246            196    196   
          mg/kg                 209            209            160    160   
mg*h/l/kg mg                    209            209            160    160   
          mg/kg                 209            209            160    160   

                             group_pk         ...          \
unit      unit_intervention                   ...           
mg*h/l    mg                       50         ...           
          mg/kg                    49         ...           
mg*h/l/kg mg                       49         ...           
          mg/kg                    49         ...           

                             (metabolic ratio, count)  (metabolic ratio, max)  \
unit      unit_intervention                                                     
mg*h/l    mg                                        0                       0   
          mg/kg                                     0                       0   
mg*h/l/kg mg                                        0                       0   
          mg/kg                                     0                       0   

                             (metabolic ratio, mean)  (metabolic ratio, min)  \
unit      unit_intervention                                                    
mg*h/l    mg                                       0                       0   
          mg/kg                                    0                       0   
mg*h/l/kg mg                                       0                       0   
          mg/kg                                    0                       0   

                             (metabolic ratio, unit)  (fat weight, count)  \
unit      unit_intervention                                                 
mg*h/l    mg                                       0                    8   
          mg/kg                                    0                    8   
mg*h/l/kg mg                                       0                    8   
          mg/kg                                    0                    8   

                             (fat weight, cv)  (fat weight, mean)  \
unit      unit_intervention                                         
mg*h/l    mg                                8                   8   
          mg/kg                             8                   8   
mg*h/l/kg mg                                8                   8   
          mg/kg                             8                   8   

                             (fat weight, se)  (fat weight, unit)  
unit      unit_intervention                                        
mg*h/l    mg                                8                   8  
          mg/kg                             8                   8  
mg*h/l/kg mg                                8                   8  
          mg/kg                             8                   8  

[4 rows x 266 columns]

In [33]:
caffeine_auc_inf_model = PkdbModel("caffeine_auc_inf",destination="3-inferred")
caffeine_auc_inf_model.data = caffeine_auc_inf
caffeine_auc_inf_model.save()